In [28]:
import os
import pandas as pd
import numpy as np
from numpy import radians, ndarray, sin, cos, degrees, arctan2, arcsin, tan, arccos
from google.cloud import storage
import datetime
from scipy.interpolate import CubicSpline
from multiprocessing import Pool
from pyproj import CRS
import rioxarray as rxr
import h5netcdf
# import pickle5 as pickle
import joblib
import fsspec
from datetime import date, datetime, timedelta
import time
import dask.array
import cftime
import sklearn
from sklearn.ensemble import GradientBoostingRegressor
import numexpr as ne

import warnings
warnings.filterwarnings("ignore")

# Configure arraylake/icechunk
import arraylake as al
import zarr
import xarray as xr
import icechunk
from icechunk.xarray import to_icechunk

import coiled
import dask
from dask import delayed
from dask.distributed import Client

import rasterio
from rasterio.crs import CRS
from rasterio.transform import from_origin
import s3fs
from pyproj import CRS, Transformer
from concurrent.futures import ThreadPoolExecutor, as_completed
import itertools

today = date.today()
doy = date.today().timetuple().tm_yday
print(today)

# python 3.11.13

2025-11-07


In [29]:
# # Configure arraylake
# from arraylake import Client
# aclient = al.Client()
# aclient.login()
# repo = aclient.get_repo('wisc-alive/ALIVE_v3-icechunk-lightweight-model-test')

In [30]:
# coiled.create_software_environment(
#     name="alive-gpp-env2",
#     conda="/Users/Sophie/Desktop/environment_final.yml")

In [31]:
# trained models
DSRmodel = joblib.load("/Users/Sophie/Desktop/ALIVE_ET_V3/hgbr_dsr_model_20250702.pkl")
# GPPmodel = joblib.load("/Users/Sophie/Desktop/alive_version2/GPP_GBR_2025-05-28.pkl")
LSTmodel = joblib.load("/Users/Sophie/Desktop/ALIVE_ET_V3/hgbr_lst_model_20250702.pkl")

mean_model = joblib.load("/Users/Sophie/Desktop/ALIVE_ET_V3/test models/trained_mean_models.pkl")
quantile_model = joblib.load("/Users/Sophie/Desktop/ALIVE_ET_V3/test models/trained_quantile_models.pkl")
uncertainty_model = joblib.load("/Users/Sophie/Desktop/ALIVE_ET_V3/test models/trained_random_uncertainty_models.pkl")

# targets = ['ETcea_NEON', 'Tcea_NEON', 'Ecea_NEON', 'NEE_PI', 'GPP_np', 'RECO_np']
GPPmodel = mean_model['GPP_np']
ETmodel = mean_model['ETcea_NEON']
Tmodel = mean_model['Tcea_NEON']
Emodel = mean_model['Ecea_NEON']

In [32]:
def ABIangle2LonLat(x, y, H, req, rpol, lon_0_deg):
    print('Finding latitude and lingutude of array point...')
    
    '''This function finds the latitude and longitude (degrees) of point P
    given x and y, the ABI elevation and scanning angle (radians)'''

    # Intermediate calculations
    a = np.sin(x)**2 + ( np.cos(x)**2 * ( np.cos(y)**2 + ( req**2 / rpol**2 ) * np.sin(y)**2 ) )
    b = -2 * H * np.cos(x) * np.cos(y)
    c = H**2 - req**2

    rs = ( -b - np.sqrt( b**2 - 4*a*c ) ) / ( 2 * a ) # distance from satellite point (S) to P

    Sx = rs * np.cos(x) * np.cos(y)
    Sy = -rs * np.sin(x)
    Sz = rs * np.cos(x) * np.sin(y)

    # Calculate lat and lon
    lat = np.arctan( ( req**2 / rpol**2 ) * ( Sz / np.sqrt( ( H - Sx )**2 + Sy**2 ) ) )
    lat = np.degrees(lat) #*
    lon = lon_0_deg - np.degrees( np.arctan( Sy / ( H - Sx )) )
    print('Latitude:', lat, 'Longitude:', lon)
    return (lon,lat)

In [33]:
def lcc_to_latlon(x, y):
    print('Finding latitude and lingutude of array point...')

    """
    Convert LCC-projected x and y coordinates to lat/lon grids.
    """
    # Define CRS
    lcc_proj4="+proj=lcc +lat_0=25 +lat_1=25 +lon_0=-95 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs"
    lcc_crs = CRS.from_proj4(lcc_proj4)
    wgs84_crs = CRS.from_epsg(4326)  # Lat/lon

    # Create transformer
    transformer = Transformer.from_crs(lcc_crs, wgs84_crs, always_xy=True)

    # Create 2D meshgrid from x and y
    xx, yy = np.meshgrid(x, y)

    # Transform to lon/lat
    lon, lat = transformer.transform(xx, yy)
    print('Latitude:', lat, 'Longitude:', lon)
    return (lon, lat)

In [34]:
def list_blobs(bucket_name,file_prefix):
    storage_client = storage.Client.create_anonymous_client()
    bucket = storage_client.get_bucket(bucket_name)
    blobs = storage_client.list_blobs(bucket_name, prefix = file_prefix) # list of files from that bucket & prefix

    blob_list = list(blobs)
    count = len(blob_list)
    return(count,blob_list,bucket) # number of files, list of files, bucket name

In [35]:
def get_images(product, year, day, hour, bucket_name):
    folder_path = 'ABI-L2-{product}/{year}/{day}/{hour}/'.format(product = product, year = year,
                                                                 day = day, hour = hour)
    numBlobs, productBlobs, bucket = list_blobs(bucket_name, folder_path)
    image_list = []

    for i in range(0, numBlobs):
        theBlob = productBlobs[i]
        location, blob_filename = os.path.split(theBlob.name)
        url = "https://storage.googleapis.com/gcp-public-data-goes-16/" + theBlob.name
        image_list.append(url)
    print(product,' IMAGE LIST -----------------------------------')
    print(image_list)
    return(image_list)

In [53]:
def reproject_array_to_match(data_array, crs, transform, match_ds):
    # print('Performing reprojections...')
    # Extract shape
    ny, nx = data_array.shape

    # Generate x and y coordinate arrays from the affine transform
    x_coords = transform.c + transform.a * (np.arange(nx) + 0.5)
    y_coords = transform.f + transform.e * (np.arange(ny) + 0.5)

    # Create DataArray with proper dims and coords
    da = xr.DataArray(
        data_array.values[np.newaxis, :, :],
        dims=["band", "y", "x"],
        coords={"band": [1], "y": y_coords, "x": x_coords},
        name="data_band"
    ).rio.write_crs(crs).rio.write_transform(transform)
    # print('Finished reprojections.')
    # Reproject to match the target dataset
    return da.rio.reproject_match(match_ds)

In [54]:
def calcSolar (ds, dt):
    '''
    x_grid = ds.x
    y_grid = ds.y
    print(x_grid,y_grid)

    lon, lat = lcc_to_latlon(x_grid, y_grid)
    lats = np.array(lat)
    lons = np.array(lon)
    '''
    print('Performing solar calculations...')
    x_grid = ds.x.values  # 1D
    y_grid = ds.y.values  # 1D

    lon, lat = lcc_to_latlon(x_grid, y_grid)  # Now 2D

    # Ensure NaNs are replaced
    latArray = np.nan_to_num(lat, nan=0)
    lonArray = np.nan_to_num(lon, nan=0)

    # from https://github.com/NASA-DEVELOP/dnppy/blob/master/dnppy/solar/README.md
    if __name__ == "__main__":
        my_datestamp   = dt                     # date stamp
        my_fmt         = "%Y-%m-%d %H:%M"       # datestamp format
        my_tz          = 0                      # timezone (GMT/UTC) offset
        my_lat = latArray                       # lat (N positive)
        my_lon = lonArray                       # lon (E positive)

        sc  = solar(my_lat, my_lon, my_datestamp, my_tz, my_fmt)
        SZA = sc.get_zenith()
        SAA = sc.get_azimuth()
    print('Finished solar calculations.')
    return SZA, SAA

In [55]:
class solar:
    """
    from: https://github.com/NASA-DEVELOP/dnppy/blob/master/dnppy/solar/solar.py
    author: 'Jwely'

    Object class for handling solar calculations. Many equations are taken from the
    excel sheet at this url : [http://www.esrl.noaa.gov/gmd/grad/solcalc/calcdetails.html]

    It requires a physical location on the earth and a datetime object

    :param lat:             decimal degrees latitude (float OR numpy array)
    :param lon:             decimal degrees longitude (float OR numpy array)
    :param time_zone:       float of time shift from GMT (such as "-5" for EST)
    :param date_time_obj:   either a timestamp string following fmt or a datetime obj
    :param fmt:             if date_time_obj is a string, fmt is required to interpret it
    :param slope:           slope of land at lat,lon for solar energy calculations
    :param aspect:          aspect of land at lat,lon for solar energy calculations

    An instance of this class may have the following attributes:

        =================== =========================================== ========
        attribute           description                                 type
        =================== =========================================== ========
        lat                 latitude                                    (array)
        lon                 longitude                                   (array)
        tz                  time zone                                   (scalar)
        rdt                 reference datetime object (date_time_obj)   (scalar)
        slope               slope, derivative of DEM                    (array)
        aspect              aspect (north is 0, south is 180)           (array)
        ajd                 absolute julian day                         (scalar)
        ajc                 absolute julian century                     (scalar)
        geomean_long        geometric mean longitude of the sun         (scalar)
        geomean_anom        geometric mean longitude anomaly of the sun (scalar)
        earth_eccent        eccentricity of earths orbit                (scalar)
        sun_eq_of_center    the suns equation of center                 (scalar)
        true_long           true longitude of the sun                   (scalar)
        true_anom           true longitude anomaly of the sun           (scalar)
        app_long            the suns apparent longitude                 (scalar)
        oblique_mean_elip   earth oblique mean ellipse                  (scalar)
        oblique_corr        correction to earths oblique elipse         (scalar)
        right_ascension     suns right ascension angle                  (scalar)
        declination         solar declination angle                     (scalar)
        equation_of_time    equation of time (minutes)                  (scalar)
        hour_angle_sunrise  the hour angle at sunrise                   (array)
        solar_noon          LST of solar noon                           (array)
        sunrise             LST of sunrise time                         (array)
        sunset              LST of sunset time                          (array)
        sunlight            LST fractional days of sunlight             (array)
        true_solar          LST for true solar time                     (array)
        hour_angle          total hour angle                            (array)
        zenith              zenith angle                                (array)
        elevation           elevation angle                             (array)
        azimuth             azimuthal angle                             (array)
        rad_vector          radiation vector (distance in AU)           (scalar)
        earth_distance      earths distance to sun in meters            (scalar)
        norm_irradiance     incident solar energy at earth distance     (scalar)
        =================== =========================================== ========

    Units used by this class unless otherwise labeled

      - angle =     degrees
      - distance =  meters
      - energy =    watts or joules
      - time =      mostly in datetime objects. labeled in most cases.

    """

    def __init__(self, lat, lon, date_time_obj, time_zone = 0,
                         fmt = False, slope = None, aspect = None):
        """
        Initializes critical spatial and temporal information for solar object.
        """

        # empty list of class attributes
        self.ajc                = None          # abs julian century (defined on __init__)
        self.ajd                = None          # abs julian day (defined on __init__)
        self.app_long           = None
        self.atmo_refraction    = None
        self.azimuth            = None
        self.declination        = None
        self.earth_distance     = None
        self.earth_eccent       = None
        self.elevation          = None
        self.elevation_noatmo   = None
        self.equation_of_time   = None
        self.frac_day           = None
        self.geomean_anom       = None
        self.geomean_long       = None
        self.hour_angle         = None
        self.hour_angle_sunrise = None
        self.lat                = lat           # lattitude (E positive)- float
        self.lat_r              = radians(lat)  # lattitude in radians
        self.lon                = lon           # longitude (N positive)- float
        self.lon_r              = radians(lon)  # longitude in radians
        self.norm_irradiance    = None
        self.oblique_corr       = None
        self.oblique_mean_elip  = None
        self.rad_vector         = None
        self.rdt                = None          # reference datetime (defined on __init__)
        self.right_ascension    = None
        self.solar_noon         = None
        self.solar_noon_time    = None
        self.sun_eq_of_center   = None
        self.sunlight           = None
        self.sunlight_time      = None
        self.sunrise            = None
        self.sunrise_time       = None
        self.sunset             = None
        self.sunset_time        = None
        self.true_anom          = None
        self.true_long          = None
        self.true_solar         = None
        self.true_solar_time    = None
        self.tz                 = None          # time zone (defined on __init__)
        self.zenith             = None

        # slope and aspect
        self.slope = slope
        self.aspect = aspect

        # Constants as attributes
        self.sun_surf_rad       = 63156942.6    # radiation at suns surface (W/m^2)
        self.sun_radius         = 695800000.    # radius of the sun in meters
        self.orbital_period     = 365.2563630   # num of days it takes earth to revolve
        self.altitude           = -0.01448623   # altitude of center of solar disk


        # sets up the object with some subfunctions
        self._set_datetime(date_time_obj, fmt, GMT_hour_offset = time_zone)

        # specify if attributes are scalar floats or numpy array floats
        if isinstance(lat, ndarray) and isinstance(lon, ndarray):
            self.is_numpy   = True
        else:
            self.is_numpy   = False

        return

    def _set_datetime(self, date_time_obj, fmt = False, GMT_hour_offset = 0):
        """
        sets the critical time information including absolute julian day/century.
        Accepts datetime objects or a datetime string with format

        :param date_time_obj:   datetime object for time of solar calculations. Will also
                                accept string input with matching value for "fmt" param
        :param fmt:             if date_time_obj is input as a string, fmt allows it to be
                                interpreted
        :param GMT_hour_offset: Number of hours from GMT for timezone of calculation area.
        """

        # if input is datetime_obj set it
        if isinstance(date_time_obj, datetime):
            self.rdt =      date_time_obj
            self.rdt +=     timedelta(hours = -GMT_hour_offset)

        elif isinstance(date_time_obj, str) and isinstance(fmt, str):
            self.rdt =      datetime.strptime(date_time_obj,fmt)
            self.rdt +=     timedelta(hours = -GMT_hour_offset)
        else:
            raise Exception("bad datetime!")

        self.tz = GMT_hour_offset

        # uses the reference day of january 1st 2000
        jan_1st_2000_jd   = 2451545
        jan_1st_2000      = datetime(2000,1,1,12,0,0)

        time_del = self.rdt - jan_1st_2000
        self.ajd = float(jan_1st_2000_jd) + float(time_del.total_seconds())/86400
        self.ajc = (self.ajd - 2451545)/36525.0

        return


    def get_geomean_long(self):
        """ :return geomean_long: geometric mean longitude of the sun"""

        if not self.geomean_long is None:
            return self.geomean_long

        self.geomean_long = (280.46646 + self.ajc * (36000.76983 + self.ajc*0.0003032)) % 360
        return self.geomean_long


    def get_geomean_anom(self):
        """calculates the geometric mean anomoly of the sun"""

        if not self.geomean_anom is None:
            return self.geomean_anom

        self.geomean_anom = (357.52911 + self.ajc * (35999.05029 - 0.0001537 * self.ajc))
        return self.geomean_anom


    def get_earth_eccent(self):
        """ :return earth_eccent: precise eccentricity of earths orbit at referece datetime """

        if not self.earth_eccent is None:
            return self.earth_eccent

        self.earth_eccent = 0.016708634 - self.ajc * (4.2037e-5 + 1.267e-7 * self.ajc)

        return self.earth_eccent


    def get_sun_eq_of_center(self):
        """ :return sun_eq_of_center: the suns equation of center"""

        if not self.sun_eq_of_center is None:
            return self.sun_eq_of_center

        if self.geomean_anom is None:
            self.get_geomean_anom()

        ajc = self.ajc
        gma = radians(self.geomean_anom)

        self.sun_eq_of_center = sin(gma) * (1.914602 - ajc*(0.004817 + 0.000014 * ajc)) + \
                                sin(2*gma) * (0.019993 - 0.000101 * ajc) + \
                                sin(3*gma) * 0.000289

        return self.sun_eq_of_center


    def get_true_long(self):
        """ :return true_long: the tru longitude of the sun"""

        if not self.true_long is None:
            return self.true_long

        if self.geomean_long is None:
            self.get_geomean_long()

        if self.sun_eq_of_center is None:
            self.get_sun_eq_of_center()

        self.true_long = self.geomean_long + self.sun_eq_of_center
        return self.true_long


    def get_app_long(self):
        """ :return app_long: calculates apparent longitude of the sun"""

        if not self.app_long is None:
            return self.app_long

        if self.true_long is None:
            self.get_true_long()

        stl = self.true_long
        ajc = self.ajc

        self.app_long = stl - 0.00569 - 0.00478 * sin(radians(125.04 - 1934.136 * ajc))
        return self.app_long


    def get_oblique_mean_elip(self):
        """ :return oblique_mean_elip: oblique mean elliptic of earth orbit """

        if not self.oblique_mean_elip is None:
            return self.oblique_mean_elip

        ajc = self.ajc

        self.oblique_mean_elip = 23 + (26 + (21.448 - ajc * (46.815 + ajc * (0.00059 - ajc * 0.001813)))/60)/60
        return self.oblique_mean_elip


    def get_oblique_corr(self):
        """ :return oblique_corr:  the oblique correction """

        if not self.oblique_corr is None:
            return self.oblique_corr

        if self.oblique_mean_elip is None:
            self.get_oblique_mean_elip()

        ome = self.oblique_mean_elip
        ajc = self.ajc

        self.oblique_corr = ome + 0.00256 * cos(radians(125.04 - 1934.136 * ajc))
        return self.oblique_corr


    def get_declination(self):
        """ :return declination: solar declination angle at ref_datetime"""

        if not self.declination is None:
            return self.declination

        if self.app_long is None:
            self.get_app_long()

        if self.oblique_corr is None:
            self.get_oblique_corr()

        sal = radians(self.app_long)
        oc  = radians(self.oblique_corr)

        self.declination = degrees(arcsin((sin(oc) * sin(sal))))
        return self.declination


    def get_equation_of_time(self):
        """ :return equation_of_time: the equation of time in minutes """

        if not self.equation_of_time is None:
            return self.equation_of_time

        if self.oblique_corr is None:
            self.get_oblique_corr()

        if self.geomean_long is None:
            self.get_geomean_long()

        if self.geomean_anom is None:
            self.get_geomean_anom()

        if self.earth_eccent is None:
            self.get_earth_eccent()

        oc  = radians(self.oblique_corr)
        gml = radians(self.geomean_long)
        gma = radians(self.geomean_anom)
        ec  = self.earth_eccent

        vary = tan(oc/2)**2

        self.equation_of_time = 4 * degrees(vary * sin(2*gml) - 2 * ec * sin(gma) +
                                4 * ec * vary * sin(gma) * cos(2 * gml) -
                                0.5 * vary * vary * sin(4 * gml) -
                                1.25 * ec * ec * sin(2 * gma))

        return self.equation_of_time


    def get_solar_noon(self):
        """ :return solar_noon: solar noon in (local sidereal time LST)"""

        if not self.solar_noon is None:
            return self.solar_noon

        if self.equation_of_time is None:
            self.get_equation_of_time()

        lon = self.lon
        eot = self.equation_of_time
        tz  = self.tz

        self.solar_noon = (720 - 4 * lon - eot + tz * 60)/1440

        # format this as a time for display purposes (Hours:Minutes:Seconds)
        if self.is_numpy:
            self.solar_noon_time = timedelta(days = self.solar_noon.mean())
        else:
            self.solar_noon_time = timedelta(days = self.solar_noon)

        return self.solar_noon


    def get_true_solar(self):
        """ :return true_solar: true solar time at ref_datetime"""

        if not self.true_solar is None:
            return self.true_solar

        if self.equation_of_time is None:
            self.get_equation_of_time()

        lon = self.lon
        eot = self.equation_of_time


        # turn reference datetime into fractional days
        frac_sec = (self.rdt - datetime(self.rdt.year, self.rdt.month, self.rdt.day)).total_seconds()
        frac_hr  = frac_sec / (60 * 60) + self.tz
        frac_day = frac_hr / 24

        self.frac_day = frac_day

        # now get true solar time
        self.true_solar = (frac_day * 1440 + eot + 4 * lon - 60 * self.tz) % 1440

        # format this as a time for display purposes (Hours:Minutes:Seconds)
        if self.is_numpy:
            self.true_solar_time = timedelta(days = self.true_solar.mean() / (60*24))
        else:
            self.true_solar_time = timedelta(days = self.true_solar / (60*24))

        return self.true_solar


    def get_hour_angle(self):
        """ :return hour_angle: returns hour angle at ref_datetime"""

        if not self.hour_angle is None:
            return self.hour_angle

        if self.true_solar is None:
            self.get_true_solar()

        ts = self.true_solar

        # matrix hour_angle calculations
        if self.is_numpy:
            ha = ts
            ha[ha <= 0] = ha[ha <= 0]/4 + 180
            ha[ha >  0] = ha[ha >  0]/4 - 180
            self.hour_angle = ha

        # scalar hour_angle calculations
        else:
            if ts <= 0:
                self.hour_angle = ts/4 + 180
            else:
                self.hour_angle = ts/4 - 180

        return self.hour_angle


    def get_zenith(self):
        """ :return zenith: returns solar zenith angle at ref_datetime"""

        if not self.zenith is None:
            return self.zenith

        if self.declination is None:
            self.get_declination()

        if self.hour_angle is None:
            self.get_hour_angle()

        d   = radians(self.declination)
        ha  = radians(self.hour_angle)
        lat = self.lat_r

        self.zenith = degrees(arccos(sin(lat) * sin(d) + cos(lat) * cos(d) * cos(ha)))

        return self.zenith


    def get_azimuth(self):
        """ :return azimuth: returns solar azimuth angle at ref_datetime"""

        if not self.azimuth is None:
            return self.azimuth

        if self.declination is None:
            self.get_declination()

        if self.hour_angle is None:
            self.get_hour_angle()

        if self.zenith is None:
            self.get_zenith()

        lat = self.lat_r
        d   = radians(self.declination)
        ha  = radians(self.hour_angle)
        z   = radians(self.zenith)

        # matrix azimuth calculations
        # these equations are hideous monsters, but im not sure how to improve them without
        # adding computational complexity.
        if self.is_numpy:

            az = ha * 0

            az[ha > 0] = (degrees(arccos(((sin(lat[ha > 0]) * cos(z[ha > 0])) - sin(d)) / (cos(lat[ha > 0]) * sin(z[ha > 0])))) + 180) % 360
            az[ha <=0] = (540 - degrees(arccos(((sin(lat[ha <=0]) * cos(z[ha <=0])) -sin(d))/ (cos(lat[ha <=0]) * sin(z[ha <=0]))))) % 360

            self.azimuth = az

        else:
            if ha > 0:
                self.azimuth = (degrees(arccos(((sin(lat) * cos(z)) - sin(d)) / (cos(lat) * sin(z)))) + 180) % 360
            else:
                self.azimuth = (540 - degrees(arccos(((sin(lat) * cos(z)) -sin(d))/ (cos(lat) * sin(z))))) % 360

        return self.azimuth

In [56]:
srf_hrrr_vars = 'SFCR', 'FRICV', 'SHTFL', 'LHTFL', 'CAPE', 'CIN', 'ULWRF', \
                'USWRF', 'DLWRF', 'DSWRF', 'VBDSF', 'VDDSF', 'HPBL', 'GFLUX','TMP'
twom_hrrr_vars = 'SPFH','RH','TMP'
hrrr_vars = srf_hrrr_vars + twom_hrrr_vars

srf_hrrr_paths = [f"surface/{var}/surface" for var in srf_hrrr_vars]
twom_hrrr_paths = [f"2m_above_ground/{var}/2m_above_ground" for var in twom_hrrr_vars]
hrrr_paths = srf_hrrr_paths + twom_hrrr_paths

In [57]:
ET_srf_hrrr_vars = 'SFCR', 'FRICV', 'LHTFL', 'CAPE', 'ULWRF', \
                'USWRF', 'DSWRF', 'VDDSF', 'HPBL', 'GFLUX','TMP'
ET_twom_hrrr_vars = 'SPFH','RH','TMP'
ET_hrrr_vars = ET_srf_hrrr_vars + ET_twom_hrrr_vars

ET_srf_hrrr_paths = [f"surface/{var}/surface" for var in ET_srf_hrrr_vars]
ET_twom_hrrr_paths = [f"2m_above_ground/{var}/2m_above_ground" for var in ET_twom_hrrr_vars]
ET_hrrr_paths = ET_srf_hrrr_paths + ET_twom_hrrr_paths

In [58]:
bucket_name = 'gcp-public-data-goes-16'
# day starts at 1 not 0

day_start = 200
day_stop = 201

hour_list = [10]
# hour_list = [0,1,10,11,12,13,14,15,16,17,18,19,20,21,22,23]
# hour_list = list(range(0,24))

year = 2024
year_str = str(year)

In [59]:
# Create list of GOES images (BRF and CMI)

bucket_name = 'gcp-public-data-goes-16' #update to goes-19 for current dates

BRFimages_all = []
CMIimages_all = []

for day in range(day_start,day_stop,1):
    day_str = str(day).zfill(3)
    print('Year: ',year,' Day: ',day_str)

    for hour in hour_list:
        hour_str = str(hour).zfill(2)
        print('Hour', hour)

        brf = get_images('BRFC', year_str, day_str, hour_str, bucket_name)
        cmi = get_images('MCMIPC', year_str, day_str, hour_str, bucket_name)

        min_len = min(len(brf), len(cmi))
        BRFimages_all.extend(brf[:min_len])
        CMIimages_all.extend(cmi[:min_len])


Year:  2024  Day:  200
Hour 10
BRFC  IMAGE LIST -----------------------------------
['https://storage.googleapis.com/gcp-public-data-goes-16/ABI-L2-BRFC/2024/200/10/OR_ABI-L2-BRFC-M6_G16_s20242001001180_e20242001003553_c20242001006319.nc', 'https://storage.googleapis.com/gcp-public-data-goes-16/ABI-L2-BRFC/2024/200/10/OR_ABI-L2-BRFC-M6_G16_s20242001006180_e20242001008553_c20242001011432.nc', 'https://storage.googleapis.com/gcp-public-data-goes-16/ABI-L2-BRFC/2024/200/10/OR_ABI-L2-BRFC-M6_G16_s20242001011180_e20242001013553_c20242001016525.nc', 'https://storage.googleapis.com/gcp-public-data-goes-16/ABI-L2-BRFC/2024/200/10/OR_ABI-L2-BRFC-M6_G16_s20242001016181_e20242001018553_c20242001021461.nc', 'https://storage.googleapis.com/gcp-public-data-goes-16/ABI-L2-BRFC/2024/200/10/OR_ABI-L2-BRFC-M6_G16_s20242001021181_e20242001023553_c20242001026284.nc', 'https://storage.googleapis.com/gcp-public-data-goes-16/ABI-L2-BRFC/2024/200/10/OR_ABI-L2-BRFC-M6_G16_s20242001026181_e20242001028553_c20242

In [60]:
print(len(BRFimages_all))

12


In [61]:
# keep only the first file in each hour
from collections import defaultdict
import re

def filter_hourly_goes_files(file_list):
    times = []
    for f in file_list:
        # Extract the "sYYYYJJJHHMMSS" pattern
        match = re.search(r's(\d{4})(\d{3})(\d{2})(\d{2})(\d{2})', f)
        if match:
            year = int(match.group(1))
            day_of_year = int(match.group(2))
            hour = int(match.group(3))
            minute = int(match.group(4))
            second = int(match.group(5))
            dt = datetime(year, 1, 1) + timedelta(days=day_of_year - 1, hours=hour, minutes=minute, seconds=second)
            times.append(dt)
        else:
            times.append(pd.NaT)

    df = pd.DataFrame({"file": file_list, "time": times}).dropna()
    df["hour"] = df["time"].dt.floor("H")

    # Keep the earliest file per hour
    df_hourly = df.sort_values("time").groupby("hour", as_index=False).first()
    return df_hourly["file"].tolist(), df_hourly["time"].tolist()

BRFimages_hourly, BRFtimes_hourly = filter_hourly_goes_files(BRFimages_all)
CMIimages_hourly, CMItimes_hourly = filter_hourly_goes_files(CMIimages_all)

# For merged time alignment
merged_datetime_range = pd.to_datetime(BRFtimes_hourly)

print('BRF images:',len(BRFimages_hourly))
print('CMI images:',len(CMIimages_hourly))
print(merged_datetime_range)


BRF images: 1
CMI images: 1
DatetimeIndex(['2024-07-18 10:01:18'], dtype='datetime64[ns]', freq=None)


In [62]:
# Create list of HRRR Images

start_date = datetime(int(year), 1, 1) + timedelta(days=int(day_start) - 1)
end_date = datetime(int(year), 1, 1) + timedelta(days=int(day_stop) - 2)

all_dates = []
current_date = start_date

while current_date <= end_date:
    all_dates.append(current_date.strftime('%Y%m%d'))
    current_date += timedelta(days=1)

HRRRimages = []

for day in all_dates: #range(day_start,day_stop,1):
    print(' Date: ',day)
    for hour in hour_list:
        hour_str = str(hour).zfill(2)
        group_url = f's3://hrrrzarr/sfc/{day}/{day}_{hour_str}z_anl.zarr/'
        print(group_url)
        HRRRimages.append(group_url)

print('HRRR images:',len(HRRRimages))

 Date:  20240718
s3://hrrrzarr/sfc/20240718/20240718_10z_anl.zarr/
HRRR images: 1


In [63]:
def load_hrrr_dataset(url):
    print('Loading HRRR dataset...')
    fs = s3fs.S3FileSystem(anon=True)
    hrrr_ds = xr.open_zarr(url, storage_options={"anon": True})

    # Step 1: Rename dimensions and assign x and y coordinates directly from chunk_index
    hrrr_ds = hrrr_ds.rename({"projection_x_coordinate": "x","projection_y_coordinate": "y"})
    hrrr_ds = hrrr_ds.assign_coords({"x": chunk_index["x"],"y": chunk_index["y"][::-1]})

    #Set spatial reference
    hrrr_ds = hrrr_ds.rio.write_crs(lambert_crs, inplace=False)
    hrrr_ds = hrrr_ds.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=False)
    hrrr_ds = hrrr_ds.set_coords(['x', 'y'])
    print('Finished loading HRRR dataset.')
    # print(hrrr_ds)
    return hrrr_ds

In [64]:
# Create projection transformations

#Open sample BRF image
brfFile = BRFimages_hourly[0]
fp2 = fsspec.open(brfFile)
brfImg = xr.open_dataset(fp2.open(), engine="h5netcdf", mask_and_scale=True)
imgTime = brfImg.t.item()
dt = datetime.utcfromtimestamp(imgTime/1000000000).strftime('%Y-%m-%d %H:%M')
day_of_year = datetime.utcfromtimestamp(imgTime/1000000000).timetuple().tm_yday

print('Performing projection transformations...')

#HRRR Projection
lambert_crs = CRS.from_proj4(
        "+proj=lcc +lat_1=38.5 +lat_2=38.5 +lat_0=38.5 +lon_0=-97.5 "
        "+x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +R=6371229")

chunk_index = xr.open_zarr("s3://hrrrzarr/grid/HRRR_chunk_index.zarr", storage_options={"anon": True})

#Open sample HRRR image
store_url = "s3://hrrrzarr/sfc/20220719/20220719_23z_anl.zarr/surface/SFCR/surface"
hrrr_ds = load_hrrr_dataset(store_url)

#GOES Projection
G16_CONUS_bounds = (-3627271.340967355, 1583173.7916531805, 1382771.9477514974, 4589199.764884492)
G16_crs = CRS.from_wkt('PROJCS["unnamed",GEOGCS["unknown",DATUM["unnamed",SPHEROID["Spheroid",6378137,298.2572221]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Geostationary_Satellite"],PARAMETER["central_meridian",-75],PARAMETER["satellite_height",35786023],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],EXTENSION["PROJ4","+proj=geos +lon_0=-75 +h=35786023 +x_0=0 +y_0=0 +ellps=GRS80 +units=m +no_defs +sweep=x"]]')
left, bottom, right, top = G16_CONUS_bounds
width, height = brfImg['BRF1'].shape[1], brfImg['BRF1'].shape[0]
transform = from_origin(left, top, (right - left) / width, -(bottom - top) / height)
print('Finished HRRR to GOES projection')

Performing projection transformations...
Loading HRRR dataset...
Finished loading HRRR dataset.
Finished HRRR to GOES projection


In [65]:
def process_image_remote(i, BRFimages, CMIimages, HRRRimages):
    imgTime = merged_datetime_range[i]
    dt = pd.to_datetime(imgTime, format='%Y-%m-%d %H:%M')  # ensures it's a Timestamp
    day_of_year = dt.timetuple().tm_yday

    if dt < pd.Timestamp("2021-01-01 00:00"):
        print(f"Skipping bad date: {dt}")
        return None

    # --- BRF ---
    brfFile = BRFimages[i]
    fp = fsspec.open(brfFile)
    print('Opening BRF file using fsspec...')
    with fp.open() as f:
        brfImg = xr.open_dataset(f, engine="h5netcdf", mask_and_scale=True)
        print('BRF image before reprojection ------------------------')
        print(brfImg)
        brf_DQF_reproj = reproject_array_to_match(
            data_array=brfImg['DQF'],
            crs=G16_crs,
            transform=transform,
            match_ds=hrrr_ds)
        print('BRF image after reprrojection ------------------------')
        print(brf_DQF_reproj)
        # imgTime = brfImg.t.item()

        # --- CMI ---
        cmiFile = CMIimages[i]
        fp2 = fsspec.open(cmiFile)
        print('Opening CMI file using fsspec...')
        with fp2.open() as f2:
            cmiImg = xr.open_dataset(f2, engine="h5netcdf", mask_and_scale=True)
            print('CMI image before reprojection ------------------------')
            print(cmiImg)
            # stack 16 CMI bands
            cmi_reprojected_bands = []
            for band_num in range(1, 17):
                band_str = str(band_num).zfill(2)
                cmi_band = cmiImg[f'CMI_C{band_str}']
                cmi_reproj = reproject_array_to_match(
                    data_array=cmi_band,
                    crs=G16_crs,
                    transform=transform,
                    match_ds=hrrr_ds)
                cmi_reprojected_bands.append(cmi_reproj.data)
            cmi_stack = np.stack(cmi_reprojected_bands, axis=-1).squeeze()
            print('CMI image after reprrojection ------------------------')
            print(cmi_stack)

            # --- Solar + DOY ---
            print('Performing solar calculations...')
            SZA_SAA = calcSolar(cmi_reproj, dt)
            SZAarray, SAAarray = SZA_SAA
            DOY = np.full((1059, 1799), day_of_year, dtype=np.int16)

            # --- DSR ---
            print('Creating DSR array...')
            DSRpredictArray = np.zeros([1059, 1799, 8])
            DSRbandIndices = [b - 1 for b in [1, 2, 4, 6, 12, 15]]
            DSRpredictArray[:, :, 0:6] = cmi_stack[:, :, DSRbandIndices]
            DSRpredictArray[:, :, 6] = SZAarray
            DSRpredictArray[:, :, 7] = SAAarray
            print('Predicting DSR...')
            dsrArr = DSRmodel.predict(np.nan_to_num(DSRpredictArray.reshape(-1, 8)))
            dsrImg = dsrArr.reshape(1059, 1799)
            print("DSR image predicted")

            # --- LST ---
            print('Creating LST array...')
            LSTpredictArray = np.zeros([1059, 1799, 12])
            LSTbandIndices = [b - 1 for b in [7, 10, 11, 12, 13, 14, 15, 16]]
            LSTpredictArray[:, :, 0:8] = cmi_stack[:, :, LSTbandIndices]
            LSTpredictArray[:, :, 8] = SZAarray
            LSTpredictArray[:, :, 9] = SAAarray
            LSTpredictArray[:, :, 10] = DOY
            LSTpredictArray[:, :, 11] = dsrImg
            print('Predicting LST...')
            lstArr = LSTmodel.predict(np.nan_to_num(LSTpredictArray.reshape(-1, 12)))
            lstImg = lstArr.reshape(1059, 1799)
            print("LST image predicted")

            # --- ET ---
            print('Creating ET array...')
            ETpredictArray = np.zeros([1059, 1799, 30])
            brfBands = ['1', '2', '3', '5', '6']
            print('Reprojecting BRF images to match ET array...')
            for k, b in enumerate(brfBands):
                brf_band = brfImg[f'BRF{b}']
                brf_reproj = reproject_array_to_match(
                    data_array=brf_band,
                    crs=G16_crs,
                    transform=transform,
                    match_ds=hrrr_ds)
                ETpredictArray[:, :, k] = brf_reproj.data

            print('Assigning BRF, DSR, LST variables...')
            BRF2, BRF3, BRF6 = ETpredictArray[:, :, 1], ETpredictArray[:, :, 2], ETpredictArray[:, :, 4]
            DSR, LST = dsrImg, lstImg

            print('Calculating NIRvP and sNIRvP')
            NIRvP = ((BRF3 - BRF2) / (BRF3 + BRF2)) * BRF3 * DSR
            sNIRvP = ((BRF3 - BRF2) / (BRF3 + BRF2)) * ((BRF3 - BRF6) / (BRF3 + BRF6)) * BRF3 * DSR
            print('NIRvP:',NIRvP,'sNIRvP:',sNIRvP)

            ETpredictArray[:, :, 5] = DSR
            ETpredictArray[:, :, 6] = LST
            ETpredictArray[:, :, 7] = NIRvP
            ETpredictArray[:, :, 8] = sNIRvP
            ETpredictArray[:, :, 9:16] = cmi_stack[:, :, [b - 1 for b in [1, 3, 12, 13, 14, 15, 16]]]

            # HRRR vars
            print('Loading and assigning HRRR data...')
            for n in range(16, 30):
                print(ET_hrrr_paths[n - 16])
                ds = load_hrrr_dataset(HRRRimages[i] + ET_hrrr_paths[n - 16])
                ETpredictArray[:, :, n] = ds[ET_hrrr_vars[n - 16]].data

            print('Reshaping ET array and prediction...')
            reshapedArr = np.reshape(ETpredictArray, (ETpredictArray.shape[0] * ETpredictArray.shape[1], ETpredictArray.shape[2]))
            maskedArr = np.nan_to_num(reshapedArr)
            etArr = ETmodel.predict(maskedArr).astype(np.float16)
            etImg = etArr.reshape(ETpredictArray[:, :, 0].shape)
            print(f"ET image {i} predicted")

            print('Creating DQF mask...')
            etImg_masked = np.where((brf_DQF_reproj[0] > 16) | np.isnan(brf_DQF_reproj[0]), 0, etImg)

            print('Creating xr dataset...')
            ds_ALIVE_ET = xr.Dataset({
                "ET": (("y", "x"), etImg_masked.astype(np.float32)),
                "ET_DQF": (("y", "x"), np.nan_to_num(brf_DQF_reproj[0], nan=255).astype(np.uint8))})
            print('Mean', np.mean(ds_ALIVE_ET))

            timestamp = pd.Timestamp(dt)
            print('Writing to timestamp ',timestamp)
            ds_to_write = ds_ALIVE_ET.expand_dims(dim={"t": [timestamp]}).reset_coords(drop=True)
            # ds_to_write = ds_ALIVE_ET.expand_dims("t").reset_coords(drop=True)
            
            ds_to_write = ds_to_write.assign_coords(t=("t", [pd.Timestamp(dt)]))
            print('ET Array ------------------------------')
            print(ds_to_write)
            return(ds_to_write)
            
            # to_icechunk(ds_to_write, session, group="ET", region='auto') # mode="a", append_dim="t")
            # return(session)



In [ ]:
# --- GPP ---
                # GPPpredictArray = np.zeros([1059, 1799, 9])
                # for k, b in enumerate(brfBands):
                #     GPPpredictArray[:, :, k] = brfImg[f'BRF{b}'].data

                # BRF2_gpp, BRF3_gpp, BRF6_gpp = GPPpredictArray[:, :, 1], GPPpredictArray[:, :, 2], GPPpredictArray[:, :, 4]

                # GPPpredictArray[:, :, 5] = NIRvP
                # GPPpredictArray[:, :, 6] = sNIRvP
                # GPPpredictArray[:, :, 7] = DSR
                # GPPpredictArray[:, :, 8] = LST

                # gppArr = GPPmodel.predict(np.nan_to_num(GPPpredictArray.reshape(-1, 9))).astype(np.float16)
                # gppImg = gppArr.reshape(1059, 1799)
                # gppImg_masked = np.where(brfImg['DQF'] > 16, 0, gppImg)

                # ds_ALIVE_GPP = xr.Dataset({
                #     "GPP": (("y", "x"), gppImg_masked.astype(np.float32)),
                #     "GPP_DQF": (("y", "x"), brfImg["DQF"].astype(np.uint8).data),
                # })
                # ds_ALIVE_GPP["t"] = pd.Timestamp(dt)

                # print("GPP predicted")
                # return ds_to_write#, ds_ALIVE_GPP

In [ ]:
def initialize_arraylake_group(variable):
    # Use HRRR as grid reference
    y_coords, x_coords = hrrr_ds.y.astype("float32"), hrrr_ds.x.astype("float32")
    ny, nx = len(y_coords), len(x_coords)
    nt = len(merged_datetime_range)
    print('New dates: ', nt)
    
    init_ds = xr.Dataset(
        {variable: (("t", "y", "x"), dask.array.full((nt, ny, nx), np.nan, dtype=np.float16, chunks=(1, ny, nx))),
         variable + "_DQF": (("t", "y", "x"), dask.array.zeros((nt, ny, nx), dtype=np.uint8, chunks=(1, ny, nx)))},
         coords={"t": merged_datetime_range, "y": y_coords, "x": x_coords},
    )
    init_ds = init_ds.reset_coords(names=["spatial_ref"], drop=True)

    # Initialize Arraylake group once for entire dataset (year)
    session = repo.writable_session(branch="main")
    # to_icechunk(init_ds, session, group=variable, region=None, mode="w") # for brand new or full write-over

    init_ds.to_zarr(session.store, compute=False, group=variable, mode="w", consolidated=False)
    session.commit("initialize " + variable + " store")

    print("✅ Dummy dataset initialized for " + variable)

# initialize_arraylake_group('ET')


In [ ]:
# def worker(i):
for i in range(len(BRFimages_hourly)):
    process_image_remote(i, BRFimages_hourly, CMIimages_hourly, HRRRimages)

# with ThreadPoolExecutor(max_workers=24) as executor:
#     futures = [executor.submit(worker, i) for i in range(len(BRFimages_hourly))]
#     wait(futures)

#### END OF CODE

Opening BRF file using fsspec...
BRF image before reprojection ------------------------
<xarray.Dataset> Size: 90MB
Dimensions:                                 (y: 1500, x: 2500,
                                             number_of_time_bounds: 2,
                                             number_of_image_bounds: 2,
                                             number_of_LZA_bounds: 2,
                                             number_of_SZA_bounds: 2)
Coordinates:
  * y                                       (y) float32 6kB 0.1282 ... 0.04427
  * x                                       (x) float32 10kB -0.1013 ... 0.03861
    t                                       datetime64[ns] 8B ...
    y_image                                 float32 4B ...
    x_image                                 float32 4B ...
    retrieval_local_zenith_angle            float32 4B ...
    quantitative_local_zenith_angle         float32 4B ...
    retrieval_solar_zenith_angle            float32 4B ...
    

In [ ]:
def main():
    # aclient = al.Client()
    # aclient.login()
    # repo = aclient.get_repo('wisc-alive/ALIVE_v3-icechunk-lightweight-model-test')
    # results = []

    session = repo.writable_session(branch="main")
    # forked_session = session.fork()

    def worker(i):
        args = [(i, BRFimages_hourly, CMIimages_hourly, HRRRimages)]
        return process_image_remote(args)  # returns ds_to_write

    with ThreadPoolExecutor(max_workers=4) as ex:
        results = list(ex.map(worker, range(min(len(BRFimages_hourly), len(CMIimages_hourly), len(HRRRimages)))))
    
    # repo_str = 'wisc-alive/ALIVE_v3-icechunk-lightweight-model-test'
    args = [(i, BRFimages_hourly, CMIimages_hourly, HRRRimages) for i in range(len(BRFimages_hourly))]
    results = process_image_remote.map(args)

    # ✅ Merge and commit all forked sessions
    session.merge(*results)

    session.commit(f"ALIVE upload for {year_str}-{day_str}")
    print(f"✅ Committed {len([r for r in results if r is not None])} datasets")


if __name__ == "__main__":
    main()


🔓 Successfully refreshed tokens!

> Token stored at /Users/sophie/.arraylake/token.json

╭──────────────────────────────────────────────── 👤 User Details ────────────────────────────────────────────────╮
│ Name: SOPHIE HOFFMAN                                                                                            │
│ Email: shoffman22@wisc.edu                                                                                      │
│ Id: b020a038-c0bd-45c5-a6f1-b9796432b6e4                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔓 Successfully logged in!

> Token stored at /Users/sophie/.arraylake/token.json

╭──────────────────────────────────────────────── 👤 User Details ────────────────────────────────────────────────╮
│ Name: SOPHIE HOFFMAN                                                                                            │
│ Email: shoffman22@wisc.edu                                                                                      │
│ Id: b020a038-c0bd-45c5-a6f1-b9796432b6e4                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

ValueError: not enough values to unpack (expected 4, got 1)

In [ ]:
def check_repo_contents(repo, group="ET"):
    # Open a read-only session on the main branch
    repo = aclient.get_repo('wisc-alive/ALIVE_v3-icechunk-lightweight-model-test')

    # first create a session
    session = repo.readonly_session(branch="main")
    try:
        # Open the group as an xarray dataset
        ds = xr.open_zarr(session.store, group=group, consolidated=False)

        print(f"✅ Group '{group}' successfully loaded.")
        print(f"   Variables: {list(ds.data_vars)}")
        print(f"   Dimensions: {dict(ds.dims)}")
        print(f"   Time range: {ds['t'].min().values} → {ds['t'].max().values}")
        print(f"   Number of timesteps: {len(ds['t'])}\n")

    except Exception as e:
        print(f"⚠️ Could not open group '{group}': {e}")


# Example usage
check_repo_contents(repo, group="ET")


✅ Group 'ET' successfully loaded.
   Variables: ['ET', 'ET_DQF']
   Dimensions: {'t': 69012, 'y': 1059, 'x': 1799}
   Time range: 2024-01-01T00:01:17.000000000 → 2024-12-30T23:56:17.000000000
   Number of timesteps: 69012



In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def visualize_icechunk_data(repo, group, start_time, end_time, variable=None):
    """
    Visualize data from an Icechunk repo over a time period.
    """
    # --- Open dataset from Icechunk ---
    repo = aclient.get_repo('wisc-alive/ALIVE_v3-icechunk-lightweight-model-test')
    session = repo.readonly_session(branch="main")
    ds = xr.open_zarr(session.store, group=group, consolidated=False)

    # Normalize time coordinate
    if not np.issubdtype(ds.t.dtype, np.datetime64):
        ds['t'] = pd.to_datetime(ds['t'].values)

    # Default to first variable if not specified
    if variable is None:
        variable = [v for v in ds.data_vars.keys() if not v.endswith("_DQF")][0]

    # --- Select time range ---
    start_time = pd.to_datetime(start_time)
    end_time = pd.to_datetime(end_time)
    ds_time = ds.sel(t=slice(start_time, end_time))

    if ds_time[variable].size == 0:
        print(f"⚠️ No data found between {start_time} and {end_time}")
        print(f"Available times: {ds.t.values[:5]} ...")
        return

    # --- Compute spatial mean over time ---
    mean_series = ds_time[variable].mean(dim=["x", "y"])

    # --- Plot time series ---
    plt.figure(figsize=(10, 4))
    mean_series.plot(marker='o', linewidth=1)
    plt.title(f"Temporal Mean of {variable} ({start_time:%Y-%m-%d} → {end_time:%Y-%m-%d})")
    plt.xlabel("Time")
    plt.ylabel(variable)
    plt.grid(True)
    plt.show()

    # --- Plot a sample map safely (use nearest match) ---
    mid_time = start_time + (end_time - start_time) / 2
    nearest_t = ds.t.sel(t=mid_time, method="nearest").item()

    plt.figure(figsize=(6, 5))
    ds[variable].sel(t=nearest_t, method="nearest").plot(cmap="viridis")
    plt.title(f"{variable} near {pd.to_datetime(str(nearest_t)):%Y-%m-%d %H:%M}")
    plt.show()

    print(f"✅ Displayed {len(ds_time.t)} timesteps from {start_time} to {end_time} for {variable}")



In [ ]:
repo = aclient.get_repo('wisc-alive/ALIVE_v3-icechunk-lightweight-model-test')
session = repo.readonly_session(branch="main")
ds = xr.open_zarr(session.store, group='ET', consolidated=False)


print(f"Number of timestamps: {len(ds.t)}")
print(f"Unique timestamps: {len(np.unique(ds.t))}")
duplicates = ds.t.to_series().duplicated(keep=False)
print(ds.t.values[duplicates.values][:10])

visualize_icechunk_data(repo, group="ET", start_time=merged_datetime_range[0], end_time=merged_datetime_range[-1])


Number of timestamps: 48
Unique timestamps: 32
['2024-01-02T00:01:17.000000000' '2024-01-02T01:01:17.000000000'
 '2024-01-02T10:01:17.000000000' '2024-01-02T11:01:17.000000000'
 '2024-01-02T12:01:17.000000000' '2024-01-02T13:01:17.000000000'
 '2024-01-02T14:01:17.000000000' '2024-01-02T15:01:17.000000000'
 '2024-01-02T16:01:17.000000000' '2024-01-02T17:01:17.000000000']


KeyError: "Cannot get left slice bound for non-unique label: Timestamp('2024-01-02 00:01:17')"